In [1]:
import json
from collections import defaultdict

import polars as pl
from scipy.interpolate import CubicSpline

In [2]:
# Cubic spline from Blue Marble Next Gen
cs = CubicSpline([0,0.25,0.75,1], [0, 179, 240, 255])

def to_rgb(dataframe, groupby):
    groupby = set(groupby)
    later_select = groupby-{"band"}
    means = dataframe.group_by(groupby).agg(pl.col.meanReflectance.mean()).drop_nans()
    return (
        means
            .with_columns(
                hex=pl.col.meanReflectance
                    .map_elements(lambda meanRefl: float(cs((meanRefl * 0.0001))), return_dtype=pl.Float32)
                    .cast(pl.UInt8, strict=False)
            )
            .drop_nulls()
            .pivot(index=later_select, values="hex", on="band").select(*later_select, "red", "green", "blue")
    )

In [3]:
most_data = pl.read_csv("data/LandColor.csv")
forest_data = pl.read_csv("data/LandColorForest.csv")
weights = pl.read_parquet("data/weights.pq")
data = most_data.join(forest_data.select("shapeName", "date", pl.selectors.starts_with("forest_")), on=["shapeName", "date"])

In [4]:
mapping = {"grassland": "Grasslands", "savanna": "Savannas", "wetland": "Wetlands", "forest": "Forests", "builtup": "Urban Areas", "water": "Water Areas", "crops": "Agricultural Areas", "shrubs": "Shrubs"}
long = (data
    .select(pl.all().exclude([".geo", "system:index"]))
    .unpivot(index = ["date", "shapeGroup", "shapeName", "shapeType"], variable_name = "variable", value_name="meanReflectance")
    .with_columns(
        pl.from_epoch(pl.col("date"), time_unit="ms"),
        pl.col("variable").str.splitn('_',2).struct.rename_fields(["landCover","band"]),
    )
    .unnest("variable")
    .with_columns(
        pl.col.landCover.replace_strict(mapping),
        month = pl.col.date.dt.month(),
        year = pl.col.date.dt.year()
    )
    .join(weights.drop("date"), on=["year", "shapeGroup", "landCover"]).sort(["shapeGroup", "landCover", "date"])
    .filter(shapeType="ADM0")
    .with_columns(pl.col.meanReflectance.replace(0, None))
)

In [5]:
long

date,shapeGroup,shapeName,shapeType,landCover,band,meanReflectance,month,year,count,area,country_weight,country_land_cover_area,global_weight
datetime[ms],str,str,str,str,str,f64,i8,i32,f64,f64,f64,f64,f64
2001-01-01 00:00:00,"""AFG""","""Afghanistan""","""ADM0""","""Agricultural Areas""","""green""",2190.182275,1,2001,79488.341176,641816.445142,0.047653,30584.45703,0.002167
2001-01-01 00:00:00,"""AFG""","""Afghanistan""","""ADM0""","""Agricultural Areas""","""red""",2392.099491,1,2001,79488.341176,641816.445142,0.047653,30584.45703,0.002167
2001-01-01 00:00:00,"""AFG""","""Afghanistan""","""ADM0""","""Agricultural Areas""","""blue""",1833.410441,1,2001,79488.341176,641816.445142,0.047653,30584.45703,0.002167
2001-02-01 00:00:00,"""AFG""","""Afghanistan""","""ADM0""","""Agricultural Areas""","""blue""",1832.266534,2,2001,79488.341176,641816.445142,0.047653,30584.45703,0.002167
2001-02-01 00:00:00,"""AFG""","""Afghanistan""","""ADM0""","""Agricultural Areas""","""green""",2193.452812,2,2001,79488.341176,641816.445142,0.047653,30584.45703,0.002167
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2020-11-01 00:00:00,"""ZWE""","""Zimbabwe""","""ADM0""","""Wetlands""","""red""",683.697297,11,2020,1510.666667,390728.902197,0.000859,335.806986,0.000216
2020-11-01 00:00:00,"""ZWE""","""Zimbabwe""","""ADM0""","""Wetlands""","""green""",619.259722,11,2020,1510.666667,390728.902197,0.000859,335.806986,0.000216
2020-12-01 00:00:00,"""ZWE""","""Zimbabwe""","""ADM0""","""Wetlands""","""red""",647.287001,12,2020,1510.666667,390728.902197,0.000859,335.806986,0.000216


In [6]:
raw_export_with_weights = (
    long
        .pivot(index=["shapeGroup", "shapeName", "landCover", "date"], values="meanReflectance", on="band")
        .with_columns(
            year=pl.col.date.dt.year(),
        )
        .join(weights.drop("date"), on=["year", "shapeGroup", "landCover"]).sort(["shapeGroup", "landCover", "date"])
        
)
raw_export_with_weights

shapeGroup,shapeName,landCover,date,green,red,blue,year,count,area,country_weight,country_land_cover_area,global_weight
str,str,str,datetime[ms],f64,f64,f64,i32,f64,f64,f64,f64,f64
"""AFG""","""Afghanistan""","""Agricultural Areas""",2001-01-01 00:00:00,2190.182275,2392.099491,1833.410441,2001,79488.341176,641816.445142,0.047653,30584.45703,0.002167
"""AFG""","""Afghanistan""","""Agricultural Areas""",2001-02-01 00:00:00,2193.452812,2371.129479,1832.266534,2001,79488.341176,641816.445142,0.047653,30584.45703,0.002167
"""AFG""","""Afghanistan""","""Agricultural Areas""",2001-03-01 00:00:00,1364.326984,1553.542652,836.470372,2001,79488.341176,641816.445142,0.047653,30584.45703,0.002167
"""AFG""","""Afghanistan""","""Agricultural Areas""",2001-04-01 00:00:00,1390.41704,1562.20156,799.839182,2001,79488.341176,641816.445142,0.047653,30584.45703,0.002167
"""AFG""","""Afghanistan""","""Agricultural Areas""",2001-05-01 00:00:00,1559.158079,1912.167028,950.918963,2001,79488.341176,641816.445142,0.047653,30584.45703,0.002167
…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZWE""","""Zimbabwe""","""Wetlands""",2020-08-01 00:00:00,451.755147,533.52457,272.968553,2020,1510.666667,390728.902197,0.000859,335.806986,0.000216
"""ZWE""","""Zimbabwe""","""Wetlands""",2020-09-01 00:00:00,494.679556,603.81844,321.168318,2020,1510.666667,390728.902197,0.000859,335.806986,0.000216
"""ZWE""","""Zimbabwe""","""Wetlands""",2020-10-01 00:00:00,553.890404,650.789711,336.434467,2020,1510.666667,390728.902197,0.000859,335.806986,0.000216


In [7]:
raw_export_with_weights.write_parquet("data/exported_reflectances.pq")

## Sidetrack: Country lookup table

Also In the name, swapping everything which comes after a comma (e.g. Bahamas, The) to the front of the name

In [8]:
country_dicts = (
    long
    .unique("shapeGroup")
    .select("shapeGroup", "shapeName")
    .sort("shapeName")
    .with_columns(pl.col.shapeName.str.split(", ").list.reverse().list.join(" ")) # putting things after the comma first
    .to_dicts())

country_lookup = {}
for item in country_dicts:
    country_lookup[item["shapeName"]] = item["shapeGroup"]

with open("countries.json", "w") as f:
    json.dump(country_lookup, f, indent=2)

In [9]:
monthly_mean_dicts = raw_export_with_weights.drop("date").to_dicts()
with open("data/exported_reflectances.js", "w") as fp:
    fp.write("var colors = "+json.dumps(monthly_mean_dicts))

## Derive means

In [10]:
# computing average color per land cover
total_land_cover_mean = (
    long
    .group_by("shapeGroup", "landCover", "band")
    .agg(
        (pl.col.meanReflectance * pl.col.country_weight).sum() / pl.col.country_weight.filter(pl.col.meanReflectance.is_not_null()).sum(),
    )
)

In [11]:
# compute total average color, weighted by land cover fraction
total_country_mean = (
    long
    .group_by("shapeGroup", "band")
    .agg(
        (pl.col.meanReflectance * pl.col.country_weight).sum() / pl.col.country_weight.filter(pl.col.meanReflectance.is_not_null()).sum(),
    )
    .with_columns(landCover=pl.lit("Total Mean Color"))
)

##  Monthly data

In [12]:
monthly_reflectances = (
    long
    .group_by("shapeGroup", "landCover", "band", "month")
    .agg(
        (pl.col.meanReflectance * pl.col.country_weight).sum() / pl.col.country_weight.filter(pl.col.meanReflectance.is_not_null()).sum(),
    )
)

In [13]:
monthly_total_reflectances = (
    long
    .group_by("shapeGroup", "band", "month")
    .agg(
        (pl.col.meanReflectance * pl.col.country_weight).sum() / pl.col.country_weight.filter(pl.col.meanReflectance.is_not_null()).sum(),
    )
    .with_columns(landCover=pl.lit("Full Country"))
)

## Global means


In [14]:
mean_query = (pl.col.meanReflectance * pl.col.global_weight).sum() / pl.col.global_weight.filter(pl.col.meanReflectance.is_not_null()).sum()

In [15]:
# computing average color per land cover
global_land_cover_mean = (
    long
    .group_by("landCover", "band")
    .agg(mean_query)
    .with_columns(shapeGroup=pl.lit("GLOBAL"))
)

In [16]:
# compute total average color, weighted by land cover fraction
global_mean = (
    long
    .group_by("band")
    .agg(mean_query)
    .with_columns(landCover=pl.lit("All Land Cover"), shapeGroup=pl.lit("GLOBAL"))
)

In [17]:
global_monthly_reflectances = (
    long
    .group_by("landCover", "band", "month")
    .agg(mean_query)
    .with_columns(shapeGroup=pl.lit("GLOBAL"))
)

global_monthly_total_reflectances = (
    long
    .group_by("band", "month")
    .agg(mean_query)
    .with_columns(landCover=pl.lit("All Land Cover"), shapeGroup=pl.lit("GLOBAL"))
)

## Rework for export

In [18]:
concat_total = pl.concat([global_mean, global_land_cover_mean, total_land_cover_mean, total_country_mean], how="diagonal").with_columns(pl.col.meanReflectance.replace(0, None)).to_dicts()

In [19]:
concat_monthly = pl.concat([
    global_monthly_reflectances, 
    global_monthly_total_reflectances,
    monthly_reflectances, 
    monthly_total_reflectances
    ], how="diagonal").with_columns(pl.col.meanReflectance.replace(0, None))

In [23]:
# Group and pivot bands and reflectances into dict
result = (
    concat_monthly
    .group_by(["shapeGroup", "landCover", "month"])
    .agg([
        pl.col("band"),
        pl.col("meanReflectance")
    ])
    .sort("shapeGroup", "landCover", "month")
    .with_columns([
        pl.struct(["band", "meanReflectance"]).map_elements(
            lambda x: dict(zip(x["band"], x["meanReflectance"]))
        ).alias("band_reflectance")
    ])
    .select(["shapeGroup", "landCover", "month", "band_reflectance"])
)

result

/tmp/ipykernel_39978/2439891685.py:10: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  .with_columns([


shapeGroup,landCover,month,band_reflectance
str,str,i8,struct[3]
"""AFG""","""Agricultural Areas""",1,"{2316.795468,2109.43491,1770.750665}"
"""AFG""","""Agricultural Areas""",2,"{2391.546344,2212.532464,1846.258909}"
"""AFG""","""Agricultural Areas""",3,"{1617.656422,1444.377545,932.051506}"
"""AFG""","""Agricultural Areas""",4,"{1428.484038,1319.759173,746.147276}"
"""AFG""","""Agricultural Areas""",5,"{1646.291456,1414.953557,823.417359}"
…,…,…,…
"""ZWE""","""Wetlands""",8,"{485.277742,431.588257,254.372991}"
"""ZWE""","""Wetlands""",9,"{571.796955,491.168866,307.820587}"
"""ZWE""","""Wetlands""",10,"{654.733096,565.752316,347.317543}"


In [24]:
output_dicts_monthly = result.group_by("shapeGroup", "landCover").agg(pl.col.month, pl.col.band_reflectance).to_dicts()

In [25]:
# Organize the data
#total_country_output = defaultdict(lambda: defaultdict(lambda: {}))

# Organize the data
def nested_dict():
    return defaultdict(dict)

total_country_output = defaultdict(lambda: defaultdict(nested_dict))

for entry in concat_total:
    shape = entry['shapeGroup']
    land = entry['landCover']
    band = entry['band']
    reflect = entry['meanReflectance']
    total_country_output[shape]["total"][land][band] = reflect

for entry in output_dicts_monthly:
    shape = entry['shapeGroup']
    land = entry['landCover']
    reflect = entry['band_reflectance']
    total_country_output[shape]["monthly"][land] = reflect


with open("total_country_reflectance_data.json", "w") as f:
    json.dump(total_country_output, f, indent=2)